# Welcome to Banners

This notebook shows the basic workflow and explaination of core functions in Banners. It walks through the following:
- Create a LocalBanner object
- How to watch for a banner
- Wave a banner
- Recall recent banners
- Retire existing banners

## Creating a LocalBanner Object

The LocalBanner object implements a Pub/Sub pattern using json files saved to a local directory. The main design intent is to have an implmentation of BaseBanner that requires no dependencies.

You can create a LocalBanner with no arguments, but there are 3 optional parameters:
- root_path: A string wherein to store events as JSON files. This defaults to /tmp/banners.
- watch_rate: How long to sleep before scanning the directory for new events
- max_events_in_topic: How many events to keep in each topic folder before cleaning up old events.

In [ ]:
from banners import LocalBanner
import tempfile

temp_dir = tempfile.TemporaryDirectory()
banner = LocalBanner(watch_rate=1, root_path=temp_dir.name)



## Waving a Banner

"Waving" is synonymous with "publish", and indicates sending a new banner for downstream consumption. We can see this in action by showing the root_path directory contents before and after the wave function is called.

Also note, the library supports `banner.publish` for users who don't want to follow the fun theme.

The file naming convention follows the following format, which is year, month, day, milliseconds into the day. This format ensures new files are easy to sort and discover. 
```
datetime.now().strftime("%Y%m%d-%H%M%S%f")
```

In [ ]:
import os

print("Initial dir contents: ", os.listdir(banner.root_path))
banner.wave("test", {"test": "value", "func": "wave"})
print("Contents after wave: ", os.listdir(banner.root_path))
print("Test topic after wave: ", os.listdir(os.path.join(banner.root_path,"test")))

## Repeat with publish function
banner.publish("test", {"test": "value", "func": "publish"})
print("Test topic after publish: ", os.listdir(os.path.join(banner.root_path,"test")))

## Recalling Events

Each banner can "recall" events that have been previously waved with the `recall_events` function. This function loads the stored json files and returns a list of dictionaries in order of oldest to newest. By default, the function returns all of the stored events, but a positive integer can be supplied to limit to returning only the most recent N events.

In [ ]:
print("'Test' events")
print(banner.recall_events("test"))

print("\nOnly return most recent event")
print(banner.recall_events("test", num_retrieve=1))


print("\nNonexistant topic recall")
print(banner.recall_events("bad_topic"))

## Watching for Banners

"Watching" is analagous to subscribing to a topic. In this call, the user requests a topic and provides a callback function that should be run when a new banner is detected.

Under the hood, Banners creates a new thread to watch for each requested topic. There can only be one thread per topic in a given process.

The callback function should be a lightweight function that is used to signal downstream action in separate processes.

Notice, the callback function will be called twice before any new banners are waved. This is because banners that were waved previously will trigger the callback function. Users can ignore banners waved before a given time using an optional argument.

In [ ]:
import threading

def callback_func(d: dict):
    print("IN CALLBACK", d)

initial_threads = [t.name for t in threading.enumerate()]

banner.watch("test", callback_func)

after_threads = [t.name for t in threading.enumerate()]
print("Threads after 'watch': ", set(after_threads) - set(initial_threads))

## Waving a New Banner

Wave a new banner to watch the callback get called

In [ ]:
banner.wave("test", {"new banner": "new value"})

## Ignore Topics

A user may wish to ignore, or unsubscribe to a topic. This can accomplished using the `ignore` function

In [ ]:
banner.ignore("test")

after_ignore = [t.name for t in threading.enumerate()]
print("Threads after 'ignore': ", set(after_ignore) - set(initial_threads))

## Cleaning up old events

The default max number of banners in a topic is relatively small (50), but a user may want to clean up old banners that may be stale.

This is a function that may use some help. Maybe a user wants to remove only a certain existing banner, that is not yet supported.

In [ ]:
banner.retire("test", 0)

## Conclude Demo

Clean up the temporary directory where the banners are stored

In [ ]:
temp_dir.cleanup()